In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from itertools import product
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor
from xgboost import plot_importance
import time

In [3]:
df=pd.read_csv("data.csv")
df

,date_block_num,shop_id,item_id,item_cnt_month,shop_category,shop_city,item_category_id,item_name_len,category_type,category_subtype,...,item_cnt_month_lag_12,shop_block_cnt_month_lag_1,shop_block_cnt_month_lag_3,shop_block_cnt_month_lag_6,shop_block_cnt_month_lag_12,item_block_cnt_month_lag_1,item_block_cnt_month_lag_3,item_block_cnt_month_lag_6,item_block_cnt_month_lag_12,best_sell_record
0,12,2,32,1,0.114087,-1.020782,-0.422891,0.388831,-0.599206,-0.248766,...,0.0,635.0,281.0,242.0,248.0,77.0,46.0,79.0,189.0,-1.314068
1,12,2,33,1,0.114087,-1.020782,-0.311681,-0.266027,-0.599206,-0.317655,...,1.0,635.0,281.0,242.0,248.0,38.0,30.0,38.0,45.0,-1.314068
2,12,2,482,2,0.114087,-1.020782,0.000000,0.367939,0.164225,-0.252864,...,1.0,635.0,281.0,242.0,248.0,75.0,111.0,82.0,95.0,-0.945401
3,12,2,839,1,0.114087,-1.020782,0.000000,-0.621370,0.164225,-0.252864,...,1.0,635.0,281.0,242.0,248.0,38.0,45.0,35.0,69.0,-1.314068
4,12,2,1007,1,0.114087,-1.020782,0.000000,0.580442,2.764180,0.000000,...,3.0,635.0,281.0,242.0,248.0,201.0,62.0,35.0,9.0,-1.129599
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2440142,34,45,18454,0,0.114087,-0.759986,-0.405790,-0.240602,-0.537773,-0.294286,...,0.0,667.0,653.0,605.0,0.0,2.0,3.0,24.0,0.0,-0.945401
2440143,34,45,16188,0,0.114087,-0.759986,0.000000,-0.170493,2.764180,-0.282879,...,0.0,667.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,-1.571419
2440144,34,45,15757,0,0.114087,-0.759986,-0.405790,-0.578909,-0.537773,-0.294286,...,0.0,667.0,653.0,605.0,851.0,5.0,4.0,10.0,7.0,-1.314068
2440145,34,45,19648,0,0.114087,-0.759986,-0.422891,-0.249088,-0.599206,-0.248766,...,0.0,667.0,653.0,605.0,0.0,2.0,6.0,4.0,0.0,-1.571419


In [ ]:
# bỏ cột label
X_train = df[df.date_block_num < 33].drop(['item_cnt_month'], axis=1)
y_train = df[df.date_block_num < 33]['item_cnt_month']
X_val = df[df.date_block_num == 33].drop(['item_cnt_month'], axis=1)
y_val =  df[df.date_block_num == 33]['item_cnt_month']
X_test = df[df.date_block_num == 34].drop(['item_cnt_month'], axis=1)

In [ ]:
# bỏ mấy cột không cần thiết
X_train.drop('date_block_num', axis=1, inplace=True)
X_val.drop('date_block_num', axis=1, inplace=True)
X_test.drop('date_block_num', axis=1, inplace=True)

In [4]:
splits = []
for block in [27, 28, 29, 30, 31, 32]:
    train_idxs = df[df.date_block_num < block].index.values
    test_idxs = df[df.date_block_num == block].index.values
    splits.append((train_idxs, test_idxs))
splits

[(array([      0,       1,       2, ..., 1263286, 1263287, 1263288],
        dtype=int64),
  array([1263289, 1263290, 1263291, ..., 1383703, 1383704, 1383705],
        dtype=int64)),
 (array([      0,       1,       2, ..., 1383703, 1383704, 1383705],
        dtype=int64),
  array([1383706, 1383707, 1383708, ..., 1507400, 1507401, 1507402],
        dtype=int64)),
 (array([      0,       1,       2, ..., 1507400, 1507401, 1507402],
        dtype=int64),
  array([1507403, 1507404, 1507405, ..., 1634828, 1634829, 1634830],
        dtype=int64)),
 (array([      0,       1,       2, ..., 1634828, 1634829, 1634830],
        dtype=int64),
  array([1634831, 1634832, 1634833, ..., 1769718, 1769719, 1769720],
        dtype=int64)),
 (array([      0,       1,       2, ..., 1769718, 1769719, 1769720],
        dtype=int64),
  array([1769721, 1769722, 1769723, ..., 1911004, 1911005, 1911006],
        dtype=int64)),
 (array([      0,       1,       2, ..., 1911004, 1911005, 1911006],
        dtype=in

In [ ]:
splits = []
for block in [27, 28, 29, 30, 31, 32]:
    train_idxs = df[df.date_block_num < block].index.values
    test_idxs = df[df.date_block_num == block].index.values
    splits.append((train_idxs, test_idxs))
splits


hyper_params = {'max_depth': [3, 4, 5, 6, 7, 8], 
                'subsample': np.arange(0.5, 1, 0.1), 
                'colsample_bytree': np.arange(0.5, 1, 0.1), 
                'learning_rate': [0.01, 0.1, 0.2, 0.3],
                'max_bin' : [256, 512, 1024],
                'n_estimators': [100, 500, 1000,3000],
               }


xgbr = XGBRegressor(seed = 1, tree_method = "gpu_hist") #gpu_hist
ts=time.time()
clf = RandomizedSearchCV(estimator = xgbr, 
                   param_distributions = hyper_params,
                   scoring = 'neg_root_mean_squared_error',
                   cv = splits)
ts=time.time()-ts
print(f"CV time: {ts}")

ts=time.time()
clf.fit(X_train, y_train)
y_hat=clf.predict(X_test)
ts=time.time()-ts
print(f"Fit time: {ts}")
print("Best parameters:", clf.best_params_)
print("Lowest RMSE: ", -clf.best_score_)

In [ ]:
xgb = XGBRegressor(seed = 1, 
    tree_method = "gpu_hist", #gpu_hist
    subsample = 0.9,
    max_depth = 7,
    learning_rate = 0.01,
    colsample_bytree = 0.7,
                   n_estimators=1000,
                   max_bin=512
    )
xgb.fit(
    X_train,y_train,
    eval_metric="rmse",
    eval_set=[(X_train, y_train), (X_val, y_val)],
    verbose=True,
    early_stopping_rounds = 10
    )

yhat_val_xgb = xgb.predict(X_val).clip(0, 20)
print('Valdation RMSE:', mean_squared_error(y_val, yhat_val_xgb, squared=False))
yhat_test_xgb = xgb.predict(X_test).clip(0, 20)

In [ ]:
def plot_features(booster, figsize):    
    fig, ax = plt.subplots(1,1,figsize=figsize)
    return plot_importance(booster=booster, ax=ax)

plot_features(xgb, (10,14))

In [ ]:
test_index=np.arange(0,214200,1)
submission = pd.DataFrame({"ID": test_index,"item_cnt_month": yhat_test_xgb})
submission.to_csv('submission_xgb.csv', index=False)